In [ ]:
!unzip -o '/content/manglish_model_retuned_3.zip' -d '/content/manglish_model_retuned_3/'

Archive:  /content/manglish_model_retuned_3.zip
   creating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/README.md  
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/adapter_model.safetensors  
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/adapter_config.json  
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/tokenizer_config.json  
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/special_tokens_map.json  
  inflating: /content/manglish_model_retuned_3/content/manglish_model_retuned_3/tokenizer.json  


In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install unsloth_zoo

In [ ]:
import os

source_dir = '/content/manglish_model_retuned_3/content/manglish_model_retuned_3'

dest_dir = '/content/manglish_model_retuned_3_flat/'
!mv "{source_dir}" "{dest_dir}"

In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
from unsloth import FastLanguageModel

app = FastAPI()

class GenerationRequest(BaseModel):
    query: str

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="manglish_model_retuned_3",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

@app.post("/generate")
def generate_text(request: GenerationRequest):
    inputs = tokenizer(
	[
    	alpaca_prompt.format(
        	"""You are a Manglish rewriter. Rewrite the Input text without explanation. Do NOT change the meaning and keep it short.
        	Example 1: "How are you?" -> "You how ah?"
        	Example 2: "I am going home." -> "I balik dulu."
        	Example 3: "Wait for me." -> "Wait ah.
        	""", # Instruction
        	f"""{request.query}""", # Input
        	"", # output - leave this blank
    	)
	], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = text.split("### Response:")[-1].strip()

    stop_words = ["Explanation:", "Note:", "###", "The original", "Meaning:"]
    for word in stop_words:
        response = response.split(word)[0].strip()

    response = response.replace('"', '').strip()
    return {"response": response}


Writing main.py


In [1]:
!pip install fastapi uvicorn pyngrok unsloth torch cloudflared
!pip install cloudflared

In [ ]:
from pyngrok import ngrok

# Change YOUR_NGROK_AUTHTOKEN to your ngrok Authtoken
ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

In [3]:
import threading
import os

def run_uvicorn():
    os.system("uvicorn main:app --host 0.0.0.0 --port 8000")

# Start both in threads
threading.Thread(target=run_uvicorn).start()
#threading.Thread(target=run_cloudflared).start()

In [4]:
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://tabatha-bribeable-gena.ngrok-free.dev" -> "http://localhost:8000"
